## import libraries

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import random
from datetime import datetime

from scripts.ml_method import *
from scripts.proj1_helpers import *
from scripts.preprocess import *
%load_ext autoreload
%autoreload 2



## Load data

In [47]:
y, x, ind = load_csv_data('higgs-data/train.csv')

## Clean data and normalize data

In [48]:
stand_x = standardize(x, with_ones= False)

In [26]:
ts = np.array([-1, 1,2,3])
loss = sum([0 if i < 0 else i for i in ts])


6

In [49]:
def hinge_loss(y, tx, w):
    tmp = 1 - y * (tx @ w)
    loss = sum([0 if i < 0 else i for i in tmp])
    return loss
    
def calculate_primal_objective(y, tx, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    v = hinge_loss(y, tx, w)
    return hinge_loss(y, tx, w) + lambda_ / 2 * np.inner(w,w)

def accuracy(y1, y2):
    return np.mean(y1 == y2)

def prediction(tx, w):
    return (tx @ w > 0) * 2 - 1

def calculate_accuracy(y, tx, w):
    """compute the training accuracy on the training set (can be called for test set as well).
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    predicted_y = prediction(tx, w)
    return accuracy(predicted_y, y)

def calculate_stochastic_gradient(y, tx, w, lambda_, n):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the index of the (one) datapoint we have sampled
    """
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!
    def is_support(y_n, x_n, w):
        """a datapoint is support if max{} is not 0. """
        return y_n * x_n @ w < 1
    
    x_n, y_n = tx[n], y[n]
    grad = - y_n * x_n.T if is_support(y_n, x_n, w) else np.zeros_like(x_n.T)
    grad = np.squeeze(grad) + lambda_ * w
    return grad



In [50]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 100000
    gamma = 2
    lambda_ = 0.1
    
    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    np.random.seed(2)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n)
        w -= gamma/(it+1) * grad
        
        if it % 10000 == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration={i}, cost={c}".format(i=it, c=cost))
    
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))
    
sgd_for_svm_demo(y, stand_x)

iteration=0, cost=881271.0642500354
iteration=10000, cost=170336.6829499165
iteration=20000, cost=169824.2085728295
iteration=30000, cost=169558.95187942567
iteration=40000, cost=169366.73915279438
iteration=50000, cost=169193.22946113822
iteration=60000, cost=169072.57449530446
iteration=70000, cost=168981.20275902862
iteration=80000, cost=169027.33776161866
iteration=90000, cost=168935.58299396152
training accuracy = 0.713704


In [13]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """compute a coordinate update (closed form) for coordinate n.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_examples)
    n: the coordinate to be updated
    """        
    # calculate the update of coordinate at index=n.
    x_n, y_n = X[n], y[n]
    old_alpha_n = alpha[n]
    
    g = (y_n * x_n.dot(w) - 1)

    if old_alpha_n == 0:
        g = min(g, 0)
    elif old_alpha_n == 1.0 / lambda_:
        g = max(g, 0)
    else:
        g = g
    if g != 0:
        alpha[n] = min(
            max(old_alpha_n - g / (x_n.T.dot(x_n)), 0.0),
            1.0 / lambda_)
    
        # compute the corresponding update on the primal vector w
        w += 1.0 * (alpha[n] - old_alpha_n) * y_n * x_n
    return w

def calculate_dual_objective(y, X, w, alpha, lambda_):
    """calculate the objective for the dual problem."""
    return np.sum(alpha * (1 - y * (X @ w))) + lambda_ / 2 * np.sum(w ** 2)

def coordinate_descent_for_svm_demo(y, X, lambda_):
    max_iter = 100000
#     lambda_ = 0.01

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    np.random.seed(2)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        w = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
            
        if it % 10000 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
#             print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
#                     it, primal_value, dual_value, duality_gap))
    print("lambda: ", lambda_, " accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

# coordinate_descent_for_svm_demo(y, X)

In [14]:
for i in np.arange(0.01, 0.3, 0.01):
    coordinate_descent_for_svm_demo(y, stand_x, i)

lambda:  0.01  accuracy = 0.641036
lambda:  0.02  accuracy = 0.660976
lambda:  0.03  accuracy = 0.710516
lambda:  0.04  accuracy = 0.670832
lambda:  0.05  accuracy = 0.6965
lambda:  0.06  accuracy = 0.648532
lambda:  0.07  accuracy = 0.67762
lambda:  0.08  accuracy = 0.693692
lambda:  0.09  accuracy = 0.713344
lambda:  0.1  accuracy = 0.665812
lambda:  0.11  accuracy = 0.709404
lambda:  0.12  accuracy = 0.664776
lambda:  0.13  accuracy = 0.635652
lambda:  0.14  accuracy = 0.673464
lambda:  0.15  accuracy = 0.696288
lambda:  0.16  accuracy = 0.655624
lambda:  0.17  accuracy = 0.68912
lambda:  0.18  accuracy = 0.697264
lambda:  0.19  accuracy = 0.672344
lambda:  0.2  accuracy = 0.719452
lambda:  0.21  accuracy = 0.681232
lambda:  0.22  accuracy = 0.625396
lambda:  0.23  accuracy = 0.689524
lambda:  0.24  accuracy = 0.623544
lambda:  0.25  accuracy = 0.678592
lambda:  0.26  accuracy = 0.64668
lambda:  0.27  accuracy = 0.679472
lambda:  0.28  accuracy = 0.68716
lambda:  0.29  accuracy = 0.

## Cross validation

In [51]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

# for i = 

x_tr, x_te, y_tr, y_te = train_test_split(stand_x, y, test_size = 0.9, random_state = 0)

# x_tr.shape
# y_tr.shape

clf = svm.SVC(kernel='rbf', C=1).fit(x_tr, y_tr)
clf.score(x_te, y_te)
# --------- Split the data into 5 parts -----------

0.82526222222222223

In [8]:
x1 = stand_x.copy()
for i in range(10,30):
    stand_x = x1
    mask = stand_x == -999
    stand_x[mask] = 0

    pca = PCA(n_components = i)
    stand_x = pca.fit_transform(stand_x)
    stand_x.shape
    
    # instantiate a logistic regression model, and fit with X and y
    model = LogisticRegression()
    model = model.fit(stand_x, y)

    # check the accuracy on the training set
    print(i,",",model.score(stand_x, y))

10 , 0.726372
11 , 0.731628
12 , 0.736484
13 , 0.736544
14 , 0.740532
15 , 0.744388
16 , 0.745148
17 , 0.745284
18 , 0.751232
19 , 0.750752
20 , 0.751012
21 , 0.751012


ValueError: n_components=22 must be between 0 and n_features=21 with svd_solver='full'